# Create Marking Form
1. Convert PDF into images.
2. Highlight student's answer

In [3]:
!sudo apt-get update
!sudo apt-get - y install poppler-utils

Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Hit:7 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Fetched 336 kB in 1s (241 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libopenjp2-7 libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ip

In [16]:
# extrat file name from pdf_file
import os
pdf_file = "../data/TestScript.pdf"
file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]
base_path = "../marking_form/" + file_name
base_path_images = base_path + "/images/"
base_path_annotations = base_path+"/annotations/"
# create directory tree for base_path_images
os.makedirs(base_path_images, exist_ok=True)
os.makedirs(base_path_annotations, exist_ok=True)

## Convert PDF to JPG

In [4]:
# read pdf and convert to images
# https://stackoverflow.com/questions/46184239/how-to-convert-pdf-to-image-using-python
from pdf2image import convert_from_path
import os

pages = convert_from_path(pdf_file, 200)
# extrat file name from pdf_file
file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]

for count, page in enumerate(pages):
    page.save(f'{base_path_images}{count}.jpg', 'JPEG')

In [58]:
import base64
import json

def reload_from_file(page,path):
    # if file not exist, return empty dict
    if not os.path.exists(path):
        return []       
    with open(path, "r") as f: 
        data = json.load(f) 
    return data[str(page)] if str(page) in data else []

def update_json_file(annotations, path):
    if not os.path.exists(path):
        with open(path, "w") as f:
            json.dump(annotations, f, indent=4) 
    else:
        with open(path, "r") as f:        
            data = json.load(f)        
        merged = {**data, **annotations}
        with open(path, "w") as f:
            json.dump(merged, f, indent=4) 

def image_to_data_url(filename):
    ext = filename.split(".")[-1]
    prefix = f"data:image/{ext};base64,"
    with open(filename, "rb") as f:
        img = f.read()
    return prefix + base64.b64encode(img).decode("utf-8")

In [63]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import json
import glob
import time

page = 1
pageAndBoxingBoxes={}

files = sorted(glob.glob(base_path_images + "*.jpg"))

annotations = {}
timestr = time.strftime("%Y%m%d-%H%M%S")

w_progress = widgets.IntProgress(value=0, max=len(files), description="Progress")
annotations_path = base_path_annotations + "annotations.json"

question_widget = widgets.Text(value="", placeholder="", description="Question:")

w_bbox = BBoxWidget(
    image=image_to_data_url(files[0]),
    # classes=["question"],
)
w_bbox.attach(question_widget, name="label")
w_bbox.bboxes = reload_from_file(w_progress.value,annotations_path)

# when Skip button is pressed we move on to the next file
def on_skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = image_to_data_url(image_file)     
    w_bbox.bboxes = reload_from_file(w_progress.value, annotations_path)


w_bbox.on_skip(on_skip)

# when Submit button is pressed we save current annotations
# and then move on to the next file
def on_submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[w_progress.value] = w_bbox.bboxes
    update_json_file(annotations, annotations_path)
    # move on to the next file
    on_skip()


w_bbox.on_submit(on_submit)
w_out = widgets.Output()

def on_bbox_change(change):
    w_out.clear_output(wait=True)
    with w_out:
        print(json.dumps(change["new"], indent=4))
        pageAndBoxingBoxes[w_progress.value] = change["new"]


w_bbox.observe(on_bbox_change, names=["bboxes"])

w_container = widgets.VBox(
    [
        widgets.HBox(
            [
                question_widget            
            ]
        ),
        w_progress,
        w_bbox,
        w_out,
    ]
)
w_container